In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
from pymongo import MongoClient

client = MongoClient();

In [49]:
def verify_ucl_data(card_id, amount):
    print("into verify_ucl_data function")  
    print("card id from kafka is ", card_id)
    print("amount from kafka is ", amount)
    try:
        client = MongoClient();
        # Database Name
        db = client["CREDIT_CARD_DB"]
        
        # Collection Name
        lookupTable = db["lookup_table"]
        
        lookupValue = lookupTable.find({'card_id': 378303738095292}) #378303738095292
        #lookupValue = lookupTable.find({'card_id': card_id}) #378303738095292
        #lookupValue = col.find({'member_id': 1314074991813})
        print("lookupvalue is ", lookupValue)
        print("lookupvalue type is ", type(lookupValue))
        
        if lookupValue is None : 
            return False
        
        for data in lookupValue:
            #print("data is ", data)
            print("dataucl is ", data["ucl"])
            print("credit score is ", data["score"])
            
        if data is None : 
            return False

        
        if amount < float(data["ucl"]) and float(data["ucl"]) > 200:
            return True
        else:
            return False
    except Exception as e:
        raise Exception(e)

In [48]:
def validateFraud(amount, card_id):
    print("into validateFraud function - part1")
    status_ucl_crdScore = verify_ucl_data(card_id, amount)
    print("Status of UCL and CreditScore is ", status_ucl_crdScore)

    return status_ucl

In [50]:
from kafka import KafkaConsumer
import sys

### Setting up the Python consumer
bootstrap_servers = ['18.211.252.152:9092']
topicName = 'transactions-topic-verified'
consumer = KafkaConsumer (topicName, group_id = 'my_group_id6',bootstrap_servers = bootstrap_servers,
auto_offset_reset = 'earliest')   ## You can also set it as latest

### Reading the message from consumer
try:
    for message in consumer:
        #print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,message.offset, message.key,message.value))
        print ("%s" % (message.value))
        kafkajsonObj = json.loads(message.value)
        status = validateFraud(kafkajsonObj["amount"], kafkajsonObj["card_id"])
        print("status is ", status)
        break
except KeyboardInterrupt:
    sys.exit()

b'{"card_id":348702330256514,"member_id":37495066290,"amount":4380912,"postcode":96774,"pos_id":248063406800722,"transaction_dt":"01-03-2018 08:24:29"}'
into validateFraud function - part1
into verify_ucl_data function
card id from kafka is  348702330256514
amount from kafka is  4380912
lookupvalue is  <pymongo.cursor.Cursor object at 0x000001D4ECAF5820>
lookupvalue type is  <class 'pymongo.cursor.Cursor'>
dataucl is  12794239.889645452
credit score is  225
Status of UCL is  True
status is  True
